In [1]:
import pandas as pd
import numpy  as np

In [2]:
dataframe = pd.read_csv("https://github.com/owid/covid-19-data/raw/master/public/data/owid-covid-data.csv")

In [3]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50090 entries, 0 to 50089
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   iso_code                         49801 non-null  object 
 1   continent                        49512 non-null  object 
 2   location                         50090 non-null  object 
 3   date                             50090 non-null  object 
 4   total_cases                      49454 non-null  float64
 5   new_cases                        49229 non-null  float64
 6   new_cases_smoothed               48447 non-null  float64
 7   total_deaths                     49454 non-null  float64
 8   new_deaths                       49229 non-null  float64
 9   new_deaths_smoothed              48447 non-null  float64
 10  total_cases_per_million          49165 non-null  float64
 11  new_cases_per_million            49165 non-null  float64
 12  new_cases_smoothed

In [4]:
dataframe.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,ABW,North America,Aruba,2020-03-13,2.0,2.0,NaN,0.0,0.0,NaN,...,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
1,ABW,North America,Aruba,2020-03-19,NaN,NaN,0.286,NaN,NaN,0.0,...,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
2,ABW,North America,Aruba,2020-03-20,4.0,2.0,0.286,0.0,0.0,0.0,...,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
3,ABW,North America,Aruba,2020-03-21,NaN,NaN,0.286,NaN,NaN,0.0,...,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
4,ABW,North America,Aruba,2020-03-22,NaN,NaN,0.286,NaN,NaN,0.0,...,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN


In [5]:
# First I want to see how many total cases are recorded in this dataset
dataframe.total_cases.sum()

5861785879.0

In [6]:
# Next I am curious which country has the most cases of COVID
data_withoutworld = dataframe[
    dataframe.location != "World"
]
data_withoutworld[data_withoutworld.total_cases == data_withoutworld.total_cases.max()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
46888,USA,North America,United States,2020-10-14,7856714.0,52517.0,50728.857,215902.0,817.0,713.286,...,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.924


In [7]:
#Next I would like to see which international country (Non-USA) has the most cases of COVID
data_withoutworld_withoutusa = data_withoutworld[
    data_withoutworld.location != "United States"
]
data_withoutworld_withoutusa[data_withoutworld_withoutusa.total_cases == data_withoutworld_withoutusa.total_cases.max()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
21652,IND,Asia,India,2020-10-14,7239389.0,63509.0,68894.0,110586.0,730.0,861.571,...,6426.674,21.2,282.28,10.39,1.9,20.6,59.55,0.53,69.66,0.64


In [8]:
# I would like to see which nation has the highest extreme_poverty and what their total_cases looks like
data_poverty = data_withoutworld[
    ['location', 'total_cases', 'extreme_poverty']
]
data_poverty[data_poverty.total_cases == data_poverty[data_poverty.extreme_poverty == data_poverty.extreme_poverty.max()].total_cases.max()]

,location,total_cases,extreme_poverty
29620,Madagascar,16754.0,77.6


In [9]:
# I would like to compare the top 10 nations who have the highest cases and see their compared extreme poverty value
data_poverty[data_poverty.total_cases == data_poverty.total_cases.max()]
data_poverty = data_poverty.sort_values('total_cases', ascending=False)

data_highestcases = data_poverty[data_poverty.total_cases == data_poverty.total_cases.max()].copy()

for index,row in data_poverty.iterrows():
    if row.location not in data_highestcases.values:
        entries = []
        entry = data_poverty[(data_poverty.location == row.location) & (data_poverty.total_cases == row.total_cases) & (data_poverty.extreme_poverty == row.extreme_poverty)]
        entries.append(entry)
        data_highestcases = pd.concat([data_highestcases, pd.concat(entries)])
        if len(data_highestcases.index) > 9:
            break
print(data_highestcases)

            location  total_cases  extreme_poverty
46888  United States    7856714.0              1.2
21652          India    7239389.0             21.2
6730          Brazil    5113628.0              3.4
39012         Russia    1326178.0              0.1
9964        Colombia     924098.0              4.5
1885       Argentina     917022.0              0.6
14516          Spain     896086.0              1.0
36342           Peru     853974.0              3.5
30128         Mexico     825340.0              2.5
49093   South Africa     694537.0             18.9
